In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import math
from torch.autograd import Variable
import copy
import pandas
import Queue
import pyedflib
from os import walk

In [2]:
files = []
time = np.array([201,1401,2601,3801,5001,6201,7401,8601,9801,11001,12201,13401,14601,15801,17001,18201,19401,20601,21801,23001,24201,25401,26601,27801,29001,30201, 31401])
dirs = ["happy", "loss", "neut"]
q_data_set = Queue.Queue()
q_targets = Queue.Queue()
for k in np.arange(3):
    files = []
    for (dirpath, dirnames, filenames) in walk(dirs[k]):
        files.extend(filenames)
        break
    for instance, file  in enumerate(files):
        f_read = pyedflib.EdfReader(dirs[k] +"/"+ file)
        n = f_read.signals_in_file
        signal_labels = f_read.getSignalLabels()
        sigbufs = np.zeros((n, f_read.getNSamples()[0]))
        shape = f_read.readSignal(0).shape[0]
        data =  np.zeros((n-1, shape))
        for i in np.arange(n-1):
            data[i, :] = f_read.readSignal(i)
        i = 0
        while (i < 26 and time[i] < shape):
            q_data_set.put(copy.deepcopy(data[:, time[i]:time[i+1]]))
            q_targets.put(k)
            i = i+1

In [3]:
data_set = np.zeros((q_data_set.qsize() - 69, 62, 1200))
targets = np.zeros((q_targets.qsize() - 69))

i = 0
j = 0
while (not q_data_set.empty()):
    temp = q_data_set.get_nowait()
    temp_targ = q_targets.get_nowait()
    if (temp.shape[1] != 1200 ):
        continue
    
    data_set[j] = copy.deepcopy(temp[:62, :])
    targets[j] = copy.deepcopy(temp_targ)
    j += 1

In [4]:
indexes = np.arange(data_set.shape[0])
np.random.shuffle(indexes)

data_set = data_set[indexes]
targets = targets[indexes]

tst_data = np.zeros((data_set.shape[0]%256, 62, 1200))
tst_targs = np.zeros((data_set.shape[0]%256))

tst_data = copy.deepcopy(data_set[data_set.shape[0] - (data_set.shape[0]%256) :])
tst_targs = copy.deepcopy(targets[targets.shape[0] - (targets.shape[0]%256) :])

val_data = copy.deepcopy(data_set[:256])
val_targs = copy.deepcopy(targets[:256])


data_set = data_set[256:(data_set.shape[0] - (data_set.shape[0]%256))]
targets = targets[256:(targets.shape[0] - (targets.shape[0]%256))]

In [10]:
data_set.shape[0]/256

7

In [5]:
def get_data(dataset, targets, batch_size):
    indexes = np.arange(dataset.shape[0])
    np.random.shuffle(indexes)
    
    dataset = dataset[indexes]
    targets = targets[indexes]
    
    tr_data = np.zeros((dataset.shape[0]/batch_size, batch_size, 62, 1200))   
    tr_targs = np.zeros((dataset.shape[0]/batch_size, batch_size))
    
    for i in range (dataset.shape[0]/batch_size):
            
        tr_data[i] = copy.deepcopy(dataset[i*batch_size:(i+1)*batch_size])
        tr_targs[i] = copy.deepcopy(targets[i*batch_size: (i+1)*batch_size])
        
    return tr_data, tr_targs

In [6]:
class LikeDislikeModel(nn.Module):
    def __init__(self, 
                 in_channels,
                 out_size,
                 L_sizes = [300, 200, 300],
                 fc_size = 20,
                 n_local_pred = 3,
                 f_sizes = [3, 3, 5], 
                 channels = [24, 16, 8], 
                 strides = [2, 2, 3], 
                 use_cuda = True):
        
        super(LikeDislikeModel, self).__init__()
        
        self.in_channels = in_channels

        self.fc_size = fc_size
        self.out_size = out_size
        self.n_local_pred = n_local_pred
        self.use_cuda = use_cuda
        self.L_sizes = L_sizes
        
        self.f_sizes = f_sizes
        self.channels = channels
        self.strides = strides
        self.len_sizes = self._len_sizes()
        
        self.conv1 = nn.ModuleList()
        self.conv2 = nn.ModuleList()
        self.conv3 = nn.ModuleList()
        self.fc1 = nn.ModuleList()
        self.fc2 = nn.ModuleList()
        
        # TODO : add batchnormalization for each conv 
        self.batchnorm = nn.ModuleList()
        
        
        
        
        for i in range (n_local_pred):
            
            self.conv1.append(nn.Conv1d(in_channels, channels[0], f_sizes[0], strides[0]))
            self.conv2.append(nn.Conv1d(channels[0], channels[1], f_sizes[1], strides[1]))               
            self.conv3.append(nn.Conv1d(channels[1], channels[2], f_sizes[2], strides[2]))
            
            self.fc1.append(nn.Linear(self.len_sizes[i][-1] * self.channels[-1], fc_size))
            self.fc2.append(nn.Linear(fc_size, out_size))
            self.batchnorm.append(nn.ModuleList())
            for convNum in range(3):
                self.batchnorm[i].append(nn.BatchNorm1d(channels[convNum]))
            
        self.dropout = nn.Dropout(p=0.5)
        
    def _len_sizes(self):
        len_sizes = []
        for i in range (self.n_local_pred):
            temp = []
            for j in range (len(self.L_sizes)):
                if j == 0:
                    temp.append(int(math.floor((self.L_sizes[i] - self.f_sizes[j])/self.strides[j] + 1)))
                else:
                    temp.append(int(math.floor((temp[j-1] - self.f_sizes[j])/self.strides[j] + 1)))
            len_sizes.append(copy.deepcopy(temp))
        return len_sizes

    def preprocessing(self, input_batch):
        nns = []
        nns.append(input_batch[:, :, 200:500])
        nns.append(input_batch[:, :, 500:700])
        nns.append(input_batch[:, :, 700:1000])
        for i in range (self.n_local_pred):
            zero_mean = nns[i] - nns[i].mean()
            for n in range(input_batch.shape[0]):
                nns[i][n] = zero_mean[n, :, :]/(np.std(zero_mean[n, :, :],axis = 0))
                
            if self.use_cuda:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i]))).cuda()
            else:
                nns[i] = Variable(torch.FloatTensor(copy.deepcopy(nns[i])))
        return nns

    def createLocalNetworks(self, nns):
        raw_output = []
        for nn_num in range( self.n_local_pred):
            conv1 = self.conv1[nn_num](nns[nn_num])
            bn1 = self.batchnorm[nn_num][0](conv1)
            relu1 = F.relu(bn1)

            conv2 = self.conv2[nn_num](relu1)
            bn2 = self.batchnorm[nn_num][1](conv2)
            relu2 = F.relu(bn2)
            
            conv3 = self.conv3[nn_num](relu2)
            bn3 = self.batchnorm[nn_num][2](conv3)
            relu3 = F.relu(bn3)
            
            relu3 = relu3.view(self.batch_size, -1)
            
            fc1 = self.fc1[nn_num](relu3)
            relu4 = F.relu(fc1)

            do = self.dropout(relu4)

            fc2 = self.fc2[nn_num](do)
            
            raw_output.append(F.log_softmax(fc2, dim = 1))
        return raw_output
    
    def _loss(self, raw_output, targets):
        
        if self.use_cuda:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_()).cuda()
        else:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_())
            
    
        for nn_num in range( self.n_local_pred):
            scores = scores + raw_output[nn_num]
        
        
        if self.use_cuda:
            targets = Variable(torch.LongTensor(targets)).cuda()
        else:
            targets = Variable(torch.LongTensor(targets))
            
        loss = F.cross_entropy(scores, targets)
        return loss
    
    def accurancy(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        
        if self.use_cuda:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_()).cuda()
        else:
            scores = Variable(torch.FloatTensor(self.batch_size, self.out_size).zero_())
            
    
        for nn_num in range( self.n_local_pred):
            scores = scores + raw_output[nn_num]
            
        _, index = torch.max(scores,1)
        index = index.cpu().data.numpy()
        acc = (index == targets).mean()
        return acc
    
    def forward(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        loss = self._loss(raw_output, targets)
        return loss

In [7]:
logdir_root = './logdir'
model = LikeDislikeModel(62, 3, use_cuda = True)

if model.use_cuda:
    model.cuda()
#model.load_state_dict(torch.load(logdir_root +  '/block99-loss=0.365_model.txt'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.00002)

In [9]:
lr = 0.00002

In [ ]:
logdir_root = './logdir2'
LossFile = open(logdir_root+'/loss.txt', 'w')
ValLossFile = open(logdir_root+'/val_loss.txt', 'w')

iterations = 7
epochs = 10000

log_loss = np.zeros(epochs*iterations)
block = 0

for epoch in range(epochs):
    tr_data, tr_targs = get_data(data_set, targets, 256)
    indexes = np.arange(iterations)
    np.random.shuffle(indexes)
    tr_data = tr_data[indexes]
    tr_targs = tr_targs[indexes]
    for iteration in range(iterations):
        batch = tr_data[iteration]
        b_targets = tr_targs[iteration]
        b_indexes = np.arange(256)
        np.random.shuffle(b_indexes)
        batch = batch[b_indexes]
        b_targets = b_targets[b_indexes]
        model.zero_grad()
        loss = model(batch, b_targets)
        log_loss[epoch*iterations + iteration] = loss.data
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        if model.use_cuda:
            v_acc = model.accurancy(val_data, val_targs)
            acc =  model.accurancy(val_data, val_targs)
            v_loss = model(batch, b_targets)
            print('block {:d}, loss = {:.3f}, loss = {:.3f}, val_acc = {:.3f}, train_acc = {:.3f}'
                  .format(block, loss.cpu().data[0], v_loss.cpu().data[0], v_acc, acc))
            ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.cpu().data[0]), 'w')
            ValLossFile.write('{:.3f} {:.3f}\n'.format(acc, v_acc))
        else:
            print('block {:d}, loss = {:.3f}'
                  .format(block, loss.data[0]))
            ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.data[0]), 'w')
            ValLossFile.write('{:.3f} {:.3f}\n'.format(loss.data[0], v_loss.data[0]))
        block += 1
    
        torch.save(model.state_dict(), ModelFile)
        ModelFile.close()
    if epoch % 400 == 0:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr/1.5)
np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')

block 0, loss = 1.048, loss = 1.051, val_acc = 0.383, train_acc = 0.434
block 1, loss = 1.054, loss = 1.049, val_acc = 0.375, train_acc = 0.379
block 2, loss = 1.044, loss = 1.047, val_acc = 0.383, train_acc = 0.438
block 3, loss = 1.043, loss = 1.042, val_acc = 0.391, train_acc = 0.406
block 4, loss = 1.035, loss = 1.045, val_acc = 0.387, train_acc = 0.406
block 5, loss = 1.009, loss = 1.031, val_acc = 0.395, train_acc = 0.371


In [69]:
np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')


In [24]:
tst_data.shape

(179, 62, 1200)

In [70]:
test = np.zeros((256, 62, 1200))
for i in range (tst_data.shape[0]):
    test[i] = tst_data[i]

In [44]:
nns = model.preprocessing(batch)
raw_output = model.createLocalNetworks(nns)
scores = Variable(torch.FloatTensor(model.batch_size, model.out_size).zero_()).cuda()
for nn_num in range( model.n_local_pred):
    scores = scores + raw_output[nn_num]

In [45]:
#scores = scores[:tst_data.shape[0]]

print (scores.shape)
_, index = torch.max(scores,1)
index = index.cpu().data.numpy()
print ((index == b_targets).mean())

torch.Size([256, 3])
0.53125


In [86]:
print (model.accurancy(tst_data, tst_targs))

[2 0 1 1 2 2 1 2 2 1 0 2 0 0 0 2 2 1 0 0 0 0 0 2 2 1 1 2 1 1 2 0 0 2 2 0 0
 0 2 0 1 1 1 0 2 1 2 1 2 1 0 2 1 0 2 0 2 0 0 1 1 0 1 1 2 2 2 1 2 1 2 1 1 2
 2 1 0 0 1 2 2 2 2 2 1 0 1 1 0 1 2 0 1 2 1 1 2 0 0 0 2 2 0 2 1 0 1 0 2 2 1
 1 1 2 0 1 2 0 0 2 1 0 0 1 1 1 0 2 1 1 1 1 1 2 2 0 1 1 2 2 0 0 1 0 1 1 1 1
 0 0 2 1 2 2 2 0 1 0 1 0 2 2 2 1 1 0 1 0 0 2 1 2 0 1 1 1 2 2 1] [ 2.  0.  1.  1.  2.  2.  1.  2.  2.  1.  0.  2.  0.  0.  0.  2.  2.  1.
  0.  0.  1.  0.  0.  2.  0.  1.  0.  2.  1.  1.  2.  0.  0.  2.  2.  0.
  0.  0.  2.  0.  1.  1.  1.  0.  2.  1.  2.  0.  1.  1.  0.  2.  1.  0.
  2.  0.  2.  0.  0.  1.  1.  0.  1.  1.  1.  0.  2.  1.  2.  1.  2.  1.
  1.  2.  2.  1.  0.  0.  1.  2.  1.  1.  2.  2.  1.  0.  1.  1.  0.  1.
  2.  0.  1.  2.  1.  1.  2.  0.  1.  0.  2.  2.  0.  2.  1.  0.  1.  0.
  2.  0.  1.  1.  1.  2.  0.  1.  2.  0.  0.  2.  1.  0.  0.  1.  1.  1.
  0.  2.  2.  1.  1.  1.  2.  2.  1.  0.  1.  1.  2.  2.  0.  0.  1.  0.
  1.  0.  1.  1.  0.  0.  2.  0.  2.  2.  2.  0.  1.